In [1]:
import pandas as pd
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier
import IPython
from sklearn import metrics 
import numpy as np
import datetime
from pandas.api.types import is_numeric_dtype
import math
from sklearn.tree import export_graphviz
import graphviz
import re
import matplotlib.pyplot as plt
import scipy
%matplotlib inline




In [2]:
PATH = "Projects/fastai/bulldoze"

In [3]:
!ls 

bulldozing_project.ipynb	  tmp		     Train.zip
Data%20Dictionary.xlsx		  Train.7z	     Valid.7z
Machine_Appendix.csv		  TrainAndValid.7z   Valid.csv
median_benchmark.csv		  TrainAndValid.csv  ValidSolution.csv
random_forest_benchmark_test.csv  TrainAndValid.zip  Valid.zip
Test.csv			  Train.csv


In [4]:
df_raw = pd.read_csv("Train.csv", low_memory=False, parse_dates = ["saledate"])

In [5]:
df_raw.head(5)

,SalesID,SalePrice,MachineID,ModelID,datasource,auctioneerID,YearMade,MachineHoursCurrentMeter,UsageBand,saledate,...,Undercarriage_Pad_Width,Stick_Length,Thumb,Pattern_Changer,Grouser_Type,Backhoe_Mounting,Blade_Type,Travel_Controls,Differential_Type,Steering_Controls
0,1139246,66000,999089,3157,121,3.0,2004,68.0,Low,2006-11-16,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Standard,Conventional
1,1139248,57000,117657,77,121,3.0,1996,4640.0,Low,2004-03-26,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Standard,Conventional
2,1139249,10000,434808,7009,121,3.0,2001,2838.0,High,2004-02-26,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1139251,38500,1026470,332,121,3.0,2001,3486.0,High,2011-05-19,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1139253,11000,1057373,17311,121,3.0,2007,722.0,Medium,2009-07-23,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [6]:
df_raw.transpose()

,0,1,2,3,4,5,6,7,8,9,...,401115,401116,401117,401118,401119,401120,401121,401122,401123,401124
SalesID,1139246,1139248,1139249,1139251,1139253,1139255,1139256,1139261,1139272,1139275,...,6333290,6333302,6333307,6333311,6333335,6333336,6333337,6333338,6333341,6333342
SalePrice,66000,57000,10000,38500,11000,26500,21000,27000,21500,65000,...,10000,8500,10000,9500,8500,10500,11000,11500,9000,7750
MachineID,999089,117657,434808,1026470,1057373,1001274,772701,902002,1036251,1016474,...,1843374,1825337,1821747,1828862,1798293,1840702,1830472,1887659,1903570,1926965
ModelID,3157,77,7009,332,17311,4605,1937,3539,36003,3883,...,21437,21437,21437,21437,21435,21439,21439,21439,21435,21435
datasource,121,121,121,121,121,121,121,121,121,121,...,149,149,149,149,149,149,149,149,149,149
auctioneerID,3,3,3,3,3,3,3,3,3,3,...,2,2,2,2,2,1,1,1,2,2
YearMade,2004,1996,2001,2001,2007,2004,1993,2001,2008,1000,...,2005,2005,2005,2006,2005,2005,2005,2005,2005,2005
MachineHoursCurrentMeter,68,4640,2838,3486,722,508,11540,4883,302,20700,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
UsageBand,Low,Low,High,High,Medium,Low,High,High,Low,Medium,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
saledate,2006-11-16 00:00:00,2004-03-26 00:00:00,2004-02-26 00:00:00,2011-05-19 00:00:00,2009-07-23 00:00:00,2008-12-18 00:00:00,2004-08-26 00:00:00,2005-11-17 00:00:00,2009-08-27 00:00:00,2007-08-09 00:00:00,...,2011-10-25 00:00:00,2011-10-25 00:00:00,2011-10-25 00:00:00,2011-10-25 00:00:00,2011-10-25 00:00:00,2011-11-02 00:00:00,2011-11-02 00:00:00,2011-11-02 00:00:00,2011-10-25 00:00:00,2011-10-25 00:00:00


In [7]:
df_raw['SalePrice'] = np.log(df_raw['SalePrice'])

In [8]:
df_raw.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 401125 entries, 0 to 401124
Data columns (total 53 columns):
SalesID                     401125 non-null int64
SalePrice                   401125 non-null float64
MachineID                   401125 non-null int64
ModelID                     401125 non-null int64
datasource                  401125 non-null int64
auctioneerID                380989 non-null float64
YearMade                    401125 non-null int64
MachineHoursCurrentMeter    142765 non-null float64
UsageBand                   69639 non-null object
saledate                    401125 non-null datetime64[ns]
fiModelDesc                 401125 non-null object
fiBaseModel                 401125 non-null object
fiSecondaryDesc             263934 non-null object
fiModelSeries               56908 non-null object
fiModelDescriptor           71919 non-null object
ProductSize                 190350 non-null object
fiProductClassDesc          401125 non-null object
state              

Extract days, month, date from "saledate" column in order to get more meaningful info from this column. 

In [9]:
def add_datepart(df, fldname, drop=True):
    fld = df[fldname]
    if not np.issubdtype(fld.dtype, np.datetime64):
        df[fldname] = fld = pd.to_datetime(fld, 
                                     infer_datetime_format=True)
    targ_pre = re.sub('[Dd]ate$', '', fldname)
    for n in ('Year', 'Month', 'Week', 'Day', 'Dayofweek', 
            'Dayofyear', 'Is_month_end', 'Is_month_start', 
            'Is_quarter_end', 'Is_quarter_start', 'Is_year_end', 
            'Is_year_start'):
        df[targ_pre+n] = getattr(fld.dt,n.lower())
    df[targ_pre+'Elapsed'] = fld.astype(np.int64) // 10**9
    if drop: df.drop(fldname, axis=1, inplace=True)

In [10]:
add_datepart(df_raw, 'saledate') 

After extracting valuable data from the 'saledate' column, we must drop this column from our pandas df if we are going to use a Random Forest Regressor Model. 

list_cols = ['year', 'month', 'isoweekday', 'Is_Weekend', 'Week', 'Day', '']

In [11]:
df_raw.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 401125 entries, 0 to 401124
Data columns (total 65 columns):
SalesID                     401125 non-null int64
SalePrice                   401125 non-null float64
MachineID                   401125 non-null int64
ModelID                     401125 non-null int64
datasource                  401125 non-null int64
auctioneerID                380989 non-null float64
YearMade                    401125 non-null int64
MachineHoursCurrentMeter    142765 non-null float64
UsageBand                   69639 non-null object
fiModelDesc                 401125 non-null object
fiBaseModel                 401125 non-null object
fiSecondaryDesc             263934 non-null object
fiModelSeries               56908 non-null object
fiModelDescriptor           71919 non-null object
ProductSize                 190350 non-null object
fiProductClassDesc          401125 non-null object
state                       401125 non-null object
ProductGroup               

There are still a fair amount of columns with non-numeric values that we will have to store as categorical variables with numeric indexes for each different value. 

In [12]:
df_subset_obj = df_raw.select_dtypes(include = ["object"])
string_cols = df_subset_obj.columns
for col in string_cols:
    df_raw[col] = pd.Categorical(df_raw[col])
df_subset_obj.head(5)


,UsageBand,fiModelDesc,fiBaseModel,fiSecondaryDesc,fiModelSeries,fiModelDescriptor,ProductSize,fiProductClassDesc,state,ProductGroup,...,Undercarriage_Pad_Width,Stick_Length,Thumb,Pattern_Changer,Grouser_Type,Backhoe_Mounting,Blade_Type,Travel_Controls,Differential_Type,Steering_Controls
0,Low,521D,521,D,NaN,NaN,NaN,Wheel Loader - 110.0 to 120.0 Horsepower,Alabama,WL,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Standard,Conventional
1,Low,950FII,950,F,II,NaN,Medium,Wheel Loader - 150.0 to 175.0 Horsepower,North Carolina,WL,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Standard,Conventional
2,High,226,226,NaN,NaN,NaN,NaN,Skid Steer Loader - 1351.0 to 1601.0 Lb Operat...,New York,SSL,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,High,PC120-6E,PC120,NaN,-6E,NaN,Small,"Hydraulic Excavator, Track - 12.0 to 14.0 Metr...",Texas,TEX,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Medium,S175,S175,NaN,NaN,NaN,NaN,Skid Steer Loader - 1601.0 to 1751.0 Lb Operat...,New York,SSL,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [13]:
df_raw.UsageBand.cat.categories

Index(['High', 'Low', 'Medium'], dtype='object')

In [14]:
df_raw.select_dtypes(include=['category'])

,UsageBand,fiModelDesc,fiBaseModel,fiSecondaryDesc,fiModelSeries,fiModelDescriptor,ProductSize,fiProductClassDesc,state,ProductGroup,...,Undercarriage_Pad_Width,Stick_Length,Thumb,Pattern_Changer,Grouser_Type,Backhoe_Mounting,Blade_Type,Travel_Controls,Differential_Type,Steering_Controls
0,Low,521D,521,D,NaN,NaN,NaN,Wheel Loader - 110.0 to 120.0 Horsepower,Alabama,WL,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Standard,Conventional
1,Low,950FII,950,F,II,NaN,Medium,Wheel Loader - 150.0 to 175.0 Horsepower,North Carolina,WL,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Standard,Conventional
2,High,226,226,NaN,NaN,NaN,NaN,Skid Steer Loader - 1351.0 to 1601.0 Lb Operat...,New York,SSL,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,High,PC120-6E,PC120,NaN,-6E,NaN,Small,"Hydraulic Excavator, Track - 12.0 to 14.0 Metr...",Texas,TEX,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Medium,S175,S175,NaN,NaN,NaN,NaN,Skid Steer Loader - 1601.0 to 1751.0 Lb Operat...,New York,SSL,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,Low,310G,310,G,NaN,NaN,NaN,Backhoe Loader - 14.0 to 15.0 Ft Standard Digg...,Arizona,BL,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,High,790ELC,790,E,NaN,LC,Large / Medium,"Hydraulic Excavator, Track - 21.0 to 24.0 Metr...",Florida,TEX,...,None or Unspecified,None or Unspecified,None or Unspecified,None or Unspecified,Double,NaN,NaN,NaN,NaN,NaN
7,High,416D,416,D,NaN,NaN,NaN,Backhoe Loader - 14.0 to 15.0 Ft Standard Digg...,Illinois,BL,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,Low,430HAG,430,HAG,NaN,NaN,Mini,"Hydraulic Excavator, Track - 3.0 to 4.0 Metric...",Texas,TEX,...,None or Unspecified,None or Unspecified,None or Unspecified,None or Unspecified,Double,NaN,NaN,NaN,NaN,NaN
9,Medium,988B,988,B,NaN,NaN,Large,Wheel Loader - 350.0 to 500.0 Horsepower,Florida,WL,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Standard,Conventional


In [15]:
df_raw.UsageBand.cat.set_categories(['High', 'Medium', 'Low'], ordered=True, inplace=True)

In [16]:
percentage_null = df_raw.isnull().sum().sort_index()/len(df_raw)
percentage_null

Backhoe_Mounting            0.803872
Blade_Extension             0.937129
Blade_Type                  0.800977
Blade_Width                 0.937129
Coupler                     0.466620
Coupler_System              0.891660
Differential_Type           0.826959
Drive_System                0.739829
Enclosure                   0.000810
Enclosure_Type              0.937129
Engine_Horsepower           0.937129
Forks                       0.521154
Grouser_Tracks              0.891899
Grouser_Type                0.752813
Hydraulics                  0.200823
Hydraulics_Flow             0.891899
MachineHoursCurrentMeter    0.644089
MachineID                   0.000000
ModelID                     0.000000
Pad_Type                    0.802720
Pattern_Changer             0.752651
ProductGroup                0.000000
ProductGroupDesc            0.000000
ProductSize                 0.525460
Pushblock                   0.937129
Ride_Control                0.629527
Ripper                      0.740388
S

In [17]:
df_raw.UsageBand.cat.codes

0         2
1         2
2         0
3         0
4         1
5         2
6         0
7         0
8         2
9         1
10        1
11        2
12       -1
13        1
14        2
15        1
16        2
17       -1
18        2
19        1
20        1
21        1
22        2
23        0
24        1
25        1
26        2
27       -1
28        2
29        1
         ..
401095   -1
401096   -1
401097   -1
401098   -1
401099   -1
401100   -1
401101   -1
401102   -1
401103   -1
401104   -1
401105   -1
401106   -1
401107   -1
401108   -1
401109   -1
401110   -1
401111   -1
401112   -1
401113   -1
401114   -1
401115   -1
401116   -1
401117   -1
401118   -1
401119   -1
401120   -1
401121   -1
401122   -1
401123   -1
401124   -1
Length: 401125, dtype: int8

In [18]:
for column in df_raw.select_dtypes(['int64']).columns:
    df_raw[column] = df_raw[column].astype('float64')

In [19]:
df_raw.dtypes

SalesID                      float64
SalePrice                    float64
MachineID                    float64
ModelID                      float64
datasource                   float64
auctioneerID                 float64
YearMade                     float64
MachineHoursCurrentMeter     float64
UsageBand                   category
fiModelDesc                 category
fiBaseModel                 category
fiSecondaryDesc             category
fiModelSeries               category
fiModelDescriptor           category
ProductSize                 category
fiProductClassDesc          category
state                       category
ProductGroup                category
ProductGroupDesc            category
Drive_System                category
Enclosure                   category
Forks                       category
Pad_Type                    category
Ride_Control                category
Stick                       category
Transmission                category
Turbocharged                category
B

In [20]:
#Functions copied from fastai.structured library
def fix_missing(df, col, name, na_dict):
    if is_numeric_dtype(col): 
        if pd.isnull(col).sum(): df[name + 'na'] = pd.isnull(col)
        df[name] = col.fillna(col.median())
        
    

In [21]:
#Also from fastai.structured
def numericalize(df, col, name, max_n_cat):
        if not is_numeric_dtype(col) and (max_n_cat is None or col.nunique()>max_n_cat):
            df[name] = col.cat.codes + 1

In [22]:
def get_sample(df, n):
    idxs = sorted(np.random.permutation(len(df))[:n])
    return df.iloc[idxs].copy()

In [23]:
#fastai.structured
def proc_df(df, y_fld=None, skip_flds=None, ignore_flds=None, do_scale=False, na_dict=None,
            preproc_fn=None, max_n_cat=None, subset=None, mapper=None):
    if not ignore_flds: ignore_flds = []
    if not skip_flds: skip_flds = []
    if subset: df = get_sample(df, subset)
    else: df = df.copy()
    ignored_flds = df.loc[:, ignore_flds]
    df.drop(ignore_flds, axis = 1, inplace=True)
    if preproc_fn: preproc_fn(df)
    if y_fld is None: y = None
    else:
        if not is_numeric_dtype(df[y_fld]): df[y_fld] = df[y_fld].cat.codes
        y = df[y_fld].values
        skip_flds += [y_fld]
    df.drop(skip_flds, axis=1, inplace=True)
    
    if na_dict is None: na_dict = {}
    else: na_dict = na_dict.copy()
    na_dict_initial = na_dict.copy()
    for n, c in df.items(): na_dict= fix_missing(df,c,n, na_dict)
    #if len(na_dict_inital.keys()) > 0:
     #   df.drop([a + '_na' for a in list(set(na_dict.keys()) - set(na_dict_initial.keys()))], axis=1, inplace=True)
    if do_scale: mapper = scale_vars(df, mapper)
    for n, c in df.items(): numericalize(df, c, n, max_n_cat)
    df = pd.get_dummies(df, dummy_na=True)
    df = pd.concat([ignored_flds, df], axis =1)
    res = [df, y, na_dict]
    if do_scale: res = res+ [mapper]
    return res
        
        

In [24]:
df, y, na_dict = proc_df(df_raw, 'SalePrice', na_dict=None)

In [25]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 401125 entries, 0 to 401124
Data columns (total 66 columns):
SalesID                       401125 non-null float64
MachineID                     401125 non-null float64
ModelID                       401125 non-null float64
datasource                    401125 non-null float64
auctioneerID                  401125 non-null float64
YearMade                      401125 non-null float64
MachineHoursCurrentMeter      401125 non-null float64
UsageBand                     401125 non-null int8
fiModelDesc                   401125 non-null int16
fiBaseModel                   401125 non-null int16
fiSecondaryDesc               401125 non-null int16
fiModelSeries                 401125 non-null int8
fiModelDescriptor             401125 non-null int16
ProductSize                   401125 non-null int8
fiProductClassDesc            401125 non-null int8
state                         401125 non-null int8
ProductGroup                  401125 non-null in

In [26]:
na_dict

In [27]:
df.columns

Index(['SalesID', 'MachineID', 'ModelID', 'datasource', 'auctioneerID',
       'YearMade', 'MachineHoursCurrentMeter', 'UsageBand', 'fiModelDesc',
       'fiBaseModel', 'fiSecondaryDesc', 'fiModelSeries', 'fiModelDescriptor',
       'ProductSize', 'fiProductClassDesc', 'state', 'ProductGroup',
       'ProductGroupDesc', 'Drive_System', 'Enclosure', 'Forks', 'Pad_Type',
       'Ride_Control', 'Stick', 'Transmission', 'Turbocharged',
       'Blade_Extension', 'Blade_Width', 'Enclosure_Type', 'Engine_Horsepower',
       'Hydraulics', 'Pushblock', 'Ripper', 'Scarifier', 'Tip_Control',
       'Tire_Size', 'Coupler', 'Coupler_System', 'Grouser_Tracks',
       'Hydraulics_Flow', 'Track_Type', 'Undercarriage_Pad_Width',
       'Stick_Length', 'Thumb', 'Pattern_Changer', 'Grouser_Type',
       'Backhoe_Mounting', 'Blade_Type', 'Travel_Controls',
       'Differential_Type', 'Steering_Controls', 'saleYear', 'saleMonth',
       'saleWeek', 'saleDay', 'saleDayofweek', 'saleDayofyear',
       'saleI

In [28]:
#after processing our dataframe , we now only have numeric values in our independent columns
df.head()

,SalesID,MachineID,ModelID,datasource,auctioneerID,YearMade,MachineHoursCurrentMeter,UsageBand,fiModelDesc,fiBaseModel,...,saleDayofyear,saleIs_month_end,saleIs_month_start,saleIs_quarter_end,saleIs_quarter_start,saleIs_year_end,saleIs_year_start,saleElapsed,auctioneerIDna,MachineHoursCurrentMeterna
0,1139246.0,999089.0,3157.0,121.0,3.0,2004.0,68.0,3,950,296,...,320.0,False,False,False,False,False,False,1.163635e+09,False,False
1,1139248.0,117657.0,77.0,121.0,3.0,1996.0,4640.0,3,1725,527,...,86.0,False,False,False,False,False,False,1.080259e+09,False,False
2,1139249.0,434808.0,7009.0,121.0,3.0,2001.0,2838.0,1,331,110,...,57.0,False,False,False,False,False,False,1.077754e+09,False,False
3,1139251.0,1026470.0,332.0,121.0,3.0,2001.0,3486.0,1,3674,1375,...,139.0,False,False,False,False,False,False,1.305763e+09,False,False
4,1139253.0,1057373.0,17311.0,121.0,3.0,2007.0,722.0,2,4208,1529,...,204.0,False,False,False,False,False,False,1.248307e+09,False,False


In [38]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 401125 entries, 0 to 401124
Data columns (total 66 columns):
SalesID                       401125 non-null float64
MachineID                     401125 non-null float64
ModelID                       401125 non-null float64
datasource                    401125 non-null float64
auctioneerID                  401125 non-null float64
YearMade                      401125 non-null float64
MachineHoursCurrentMeter      401125 non-null float64
UsageBand                     401125 non-null int8
fiModelDesc                   401125 non-null int16
fiBaseModel                   401125 non-null int16
fiSecondaryDesc               401125 non-null int16
fiModelSeries                 401125 non-null int8
fiModelDescriptor             401125 non-null int16
ProductSize                   401125 non-null int8
fiProductClassDesc            401125 non-null int8
state                         401125 non-null int8
ProductGroup                  401125 non-null in

In [42]:
df.as_matrix().shape

/home/ec2-user/anaconda3/lib/python3.6/site-packages/ipykernel/__main__.py:1: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  if __name__ == '__main__':


(401125, 66)

In [43]:
df

,SalesID,MachineID,ModelID,datasource,auctioneerID,YearMade,MachineHoursCurrentMeter,UsageBand,fiModelDesc,fiBaseModel,...,saleDayofyear,saleIs_month_end,saleIs_month_start,saleIs_quarter_end,saleIs_quarter_start,saleIs_year_end,saleIs_year_start,saleElapsed,auctioneerIDna,MachineHoursCurrentMeterna
0,1139246.0,999089.0,3157.0,121.0,3.0,2004.0,68.0,3,950,296,...,320.0,False,False,False,False,False,False,1.163635e+09,False,False
1,1139248.0,117657.0,77.0,121.0,3.0,1996.0,4640.0,3,1725,527,...,86.0,False,False,False,False,False,False,1.080259e+09,False,False
2,1139249.0,434808.0,7009.0,121.0,3.0,2001.0,2838.0,1,331,110,...,57.0,False,False,False,False,False,False,1.077754e+09,False,False
3,1139251.0,1026470.0,332.0,121.0,3.0,2001.0,3486.0,1,3674,1375,...,139.0,False,False,False,False,False,False,1.305763e+09,False,False
4,1139253.0,1057373.0,17311.0,121.0,3.0,2007.0,722.0,2,4208,1529,...,204.0,False,False,False,False,False,False,1.248307e+09,False,False
5,1139255.0,1001274.0,4605.0,121.0,3.0,2004.0,508.0,3,493,175,...,353.0,False,False,False,False,False,False,1.229558e+09,False,False
6,1139256.0,772701.0,1937.0,121.0,3.0,1993.0,11540.0,1,1453,419,...,239.0,False,False,False,False,False,False,1.093478e+09,False,False
7,1139261.0,902002.0,3539.0,121.0,3.0,2001.0,4883.0,1,740,243,...,321.0,False,False,False,False,False,False,1.132186e+09,False,False
8,1139272.0,1036251.0,36003.0,121.0,3.0,2008.0,302.0,3,779,250,...,239.0,False,False,False,False,False,False,1.251331e+09,False,False
9,1139275.0,1016474.0,3883.0,121.0,3.0,1000.0,20700.0,2,1771,540,...,221.0,False,False,False,False,False,False,1.186618e+09,False,False


In [29]:
m = RandomForestRegressor(n_jobs = -1)
m.fit(df, y)
m.score(df, y)

0.98313641904810778

Our model fit well, however we need to check for overfitting and split up our dataset into training and validation sets. 

In [30]:
def split_vals(a, n): return a[:n].copy(), a[n:].copy()
n_valid = 12000
n_trn = len(df)-n_valid
raw_train, raw_valid = split_vals(df_raw, n_trn)
X_train, X_valid = split_vals(df, n_trn)
y_train, y_valid = split_vals(y, n_trn)

X_train.shape, y_train.shape, X_valid.shape

((389125, 66), (389125,), (12000, 66))

Here is our model result with a most likely overfitting model

In [31]:
def rmse(x, y): return math.sqrt(((x-y)**2).mean())

def print_score(m):
    res = [rmse(m.predict(X_train), y_train), rmse(m.predict(X_valid), y_valid), m.score(X_train, y_train), m.score(X_valid, y_valid)]
    if hasattr(m, 'oob_score_'): res.append(m.oob_score_)
    print(res)

In [32]:
m = RandomForestRegressor(n_jobs = -1)
%time m.fit(X_train, y_train)
print_score(m)

CPU times: user 1min 12s, sys: 344 ms, total: 1min 13s
Wall time: 13.7 s
[0.09057640705330716, 0.24949281031301007, 0.98285391372569797, 0.88883603112376808]


In [33]:
df_trn, y_trn, na_dict = proc_df(df_raw, 'SalePrice', subset = 30000)
X_train, _ = split_vals(df_trn, 20000)
y_train, _ = split_vals(y_trn, 20000)

In [34]:
m = RandomForestRegressor(n_jobs = -1)
%time m.fit(X_train, y_train)
print_score(m)


CPU times: user 2.71 s, sys: 4.11 ms, total: 2.71 s
Wall time: 554 ms
[0.11207955113896194, 0.3513426048287634, 0.97244604620428987, 0.77955025155550728]


In [35]:
m = RandomForestRegressor(n_estimators = 1, max_depth =3, bootstrap=False, n_jobs = -1)
m.fit(X_train, y_train)
print_score(m)

[0.5216353670936204, 0.5818033638339496, 0.40314997285593746, 0.39549425891753953]


In [36]:
#fastai function
def draw_tree(t, df, size=10, ratio=0.6, precision=0):
    s=export_graphviz(t, out_file=None, feature_names=df.columns, filled=True,
                      special_characters=True, rotate=True, precision=precision)
    IPython.display.display(graphviz.Source(re.sub('Tree {',
       f'Tree {{ size={size}; ratio={ratio}', s)))


In [37]:
draw_tree(m.estimators_[0], df_trn, precision=3)

TypeError: export_graphviz() got an unexpected keyword argument 'precision'

In [ ]:
m = RandomForestRegressor(n_estimators =1, bootstrap=False, n_jobs=-1)
m.fit(X_train, y_train)
print_score(m)


Adapting the model to become a random forest using bagging

In [ ]:
m = RandomForestRegressor(n_jobs=-1, n_estimators=10, bootstrap=True)
m.fit(X_train,y_train)
print_score(m)

In [ ]:
preds = np.stack([t.predict(X_valid) for t in m.estimators_])
preds[:, 0], np.mean(preds[:, 0]), y_valid[0] 

In [ ]:
preds.shape

In [ ]:
plt.plot([metrics.r2_score(y_valid, np.mean(preds[:i+1], axis=0)) for i in range(10)])




Out-of-bag Score
 when the validation set is small and we do not want to random bagging because random sampling of a small data set will offer less generalization advantage due to fact that random samples are more representative of the small dataset. Higher correlation between our trees. In order to do this we calculate error on the training set but only include the trees in the calculation of a row's error that did not include that row when training that tree. Thus, can we see whether the model is overfitting.

In [ ]:
m = RandomForestRegressor(n_estimators=40, n_jobs=-1, oob_score=True)
m.fit(X_train, y_train)
print_score(m)

Grid-search - 
feed a list of potential hyperarametr values and have sklearn test all diff given combination of those hyperparameter

subsampling-different random subset per tree, that the tree will sample from. 

In [ ]:
df_trn, y_trn, na_dict = proc_df(df_raw, 'SalePrice')
X_train, X_valid = split_vals(df_trn, n_trn)
y_train, y_valid = split_vals(y_trn, n_trn)

In [ ]:
len(df_trn)

In [ ]:
#from fastai
from sklearn.ensemble import forest
#def set_rf_samples(n):
 #   forest._generate_sample_indices = (lambda rs, n_samples: forest.check_random_state(rs).randint(0, n_samples, n))
#set_rf_samples(20000)

In [ ]:
m = RandomForestRegressor(n_jobs = -1, oob_score = True, n_estimators=40)
%time m.fit(X_train, y_train)
print_score(m)

In [ ]:
??reset_rf_samples

In [ ]:
m = RandomForestRegressor(n_estimators=40, min_samples_leaf=3, n_jobs = -1, oob_score=True, max_features = 0.5) 
m.fit(X_train,y_train)
print_score(m)

In [ ]:
df_raw.fiProductClassDesc.cat.categories

Lesson 3- making sense of our data/error values/predictions

In [ ]:
m = RandomForestRegressor(n_jobs = -1)
m.fit(df, y)
m.score(df, y)


In [ ]:
df_raw.head()

In [ ]:
#fastai function
def set_rf_samples(n):
    forest._generate_sample_indices = (lambda rs, n_samples: forest.check_random_state(rs).randint(0, n_samples, n))
set_rf_samples(50000)

In [ ]:
m = RandomForestRegressor(n_estimators=40, min_samples_leaf=3, max_features=0.5, n_jobs=-1, oob_score=True)
m.fit(X_train, y_train)
print_score(m)

In [ ]:
%time preds = np.stack([t.predict(X_valid) for t in m.estimators_])
np.mean(preds[:, 0]), np.std(preds[:, 0])

In [ ]:
from concurrent.futures import ProcessPoolExecutor
def parallel_trees(m, fn, n_jobs=8):
    return list(ProcessPoolExecutor(n_jobs).map(fn, m.estimators_))

In [ ]:
def get_preds(t): return t.predict(X_valid)
%time preds = np.stack(parallel_trees(m, get_preds))
np.mean(preds[:, 0]), np.std(preds[:, 0]) 

In [ ]:
x = raw_valid.copy()
x['pred_std'] = np.std(preds, axis=0)
x['pred'] = np.mean(preds, axis=0)
x.Enclosure.value_counts().plot.barh()


In [ ]:
flds = ['Enclosure', 'SalePrice', 'pred', 'pred_std']
enc_summ = x[flds].groupby('Enclosure', as_index=False).mean()
enc_summ

Fastai function below that uses sklearn's built in functionality to determine feature importance and return a dataframe with columns and feature importance. 

In [ ]:
def rf_feat_importance(m, df):
    return pd.DataFrame({'cols':df.columns, 'imp':m.feature_importances_}
                       ).sort_values('imp', ascending=False)

In [ ]:
f1 = rf_feat_importance(m, df_trn); f1[:10]

In [ ]:
#Jermeey says he starts with this step right after creating his first several forests that are bttter than aking the average.

f1.plot('cols', 'imp',figsize=(10, 6), legend=False)

In [ ]:
f1.plot.barh('cols', 'imp', figsize=(12, 10))

Jeremy says at this point he would sit down with the domain expert/client and use this type of graph to lead questioning/information gathering. 

In [ ]:
to_keep = f1[f1.imp>0.005].cols
len(to_keep)

In [ ]:
df_keep = df_trn[to_keep].copy()
X_train, X_valid = split_vals(df_keep, n_trn)

In [ ]:
m = RandomForestRegressor(n_estimators=40, min_samples_leaf=3, max_features=0.5, n_jobs=-1, oob_score = True)
m.fit(X_train, y_train)
print_score(m)

In [ ]:
def plot_f1(f1): return f1.plot('cols', 'imp', 'barh', figsize=(12, 7), legend=False)

In [ ]:
f1 = rf_feat_importance(m, df_keep)
plot_f1(f1[:30])

In [ ]:
df_trn2, y_trn, nas = proc_df(df_raw, 'SalePrice', max_n_cat=7)
X_train, X_valid = split_vals(df_trn2, n_trn)

m = RandomForestRegressor(n_estimators=40, min_samples_leaf =3, max_features=0.6, n_jobs = -1, oob_score=True)
m.fit(X_train, y_train)
print_score(m)

In [ ]:
fi = rf_feat_importance(m, df_trn2)
plot_f1(fi[:25])

Note, now Enclosure_EROPS w AC is massively imporant after we have one hot encoded our categorical variables with less than or equal to 7 cardinality. 

In [ ]:
from scipy.cluster import hierarchy as hc

In [ ]:
corr = np.round(scipy.stats.spearmanr(df_keep).correlation, 4)
corr_condensed = hc.distance.squareform(1-corr)
z = hc.linkage(corr_condensed, method='average')
fig = plt.figure(figsize=(16, 10))
dendrogram = hc.dendrogram(z, labels = df_keep.columns, orientation='left', leaf_font_size=16)
plt.show()

In [ ]:
def get_oob(df):
    m = RandomForestRegressor(n_estimators=30, min_samples_leaf=5, max_features=0.6, n_jobs=-1, oob_score=True)
    x, _ = split_vals(df, n_trn)
    m.fit(x, y_train)
    return m.oob_score_

In [ ]:
get_oob(df_keep)

In [ ]:
for c in ('saleYear', 'saleElapsed', 'fiModelDesc','fiBaseModel', 'Grouser_Tracks', 'Coupler_System'):
    print(c, get_oob(df_keep.drop(c, axis=1)))

In [ ]:
to_drop = ['saleYear', 'fiBaseModel', 'Grouser_Tracks']
get_oob(df_keep.drop(to_drop, axis=1))

In [ ]:
df_keep.drop(to_drop, axis=1, inplace=True)
X_train, X_valid = split_vals(df_keep, n_trn)

In [ ]:
np.save('tmp/keep_cols.npy', np.array(df_keep.columns))

In [ ]:
keep_cols = np.load('tmp/keep_cols.npy')
df_keep = df_trn[keep_cols]

In [ ]:
reset_rf_samples()

In [ ]:
m = RandomForestRegressor(n_estimators=40, min_samples_leaf=3, max_features=0.5, n_jobs=-1, oob_score=True)
m.fit(X_train, y_train)
print_score(m)

Try and understand partial dependence


In [ ]:
from pdpbox import pdp
from plotnine import *



In [ ]:
df_trn2, y_trn, nas = proc_df(df_raw, 'SalePrice', max_n_cat=7)
X_train, X_valid = split_vals(df_trn2, n_trn)
m = RandomForestRegressor(n_estimators=40, min_samples_leaf=3, max_features=0.6, n_jobs=-1)
m.fit(X_train, y_train)

In [ ]:
plot_f1(rf_feat_importance(m, df_trn2)[:10])

In [ ]:
df_raw.plot('YearMade', 'saleElapsed', 'scatter', alpha=0.1, figsize=(10, 8))

In [ ]:
def get_sample(df, n):
    idxs = sorted(np.random.permutation(len(df))[:n])
    return df.iloc[idxs].copy()

In [ ]:
x_all = get_sample(df_raw[df_raw.YearMade>1930], 500)

In [ ]:
!ls

In [ ]:
#plot the partial dependence 
def plot_pdp(feat, clusters=None, feat_name=None, feat_type=None):
    feat_name = feat_name or feat
    p = pdp.pdp_isolate(m, x, feat_type, feat)
    return pdp.pdp_plot(p, feat_name, plot_lines=True, cluster=clusters is not None, n_cluster_centers=clusters)

In [ ]:
plot_pdp('YearMade')

In [ ]:
p = pdp.pdp_isolate(m, X_train, df_trn2.columns, 'YearMade')
pdp.pdp_plot(p, 'YearMade', plot_lines=True, cluster=clusters is not None, n_cluster_centers=clusters)

In [ ]:
df_raw.YearMade[df_raw.YearMade<1950] = 1950
df_keep['age'] = df_raw['age'] = df_raw.saleYear-df_raw.YearMade

In [ ]:
X_train, X_valid = split_vals(df_keep, n_trn)
m = RandomFor